In [1]:
import math
import numpy as np 
import matplotlib.pyplot as plt
import random
import graphviz
%matplotlib inline

In [15]:
class Value:
    def __init__(self, data, _children = (), _op = '', label = ''):
        self.data = data
        self.grad = 0.0
        self._backward = lambda : None
        self._prev = set(_children)
        self._op = _op
        self.label = label

    def __repr__(self):
        return f"Value(data={self.data})"

    def __add__(self, other: 'Value'):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), '+')

        def _backward():
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad

        out._backward= _backward
        return out

    def __radd__(self, other):
        return self + other

    def __mul__(self, other: 'Value'):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), '*')

        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad

        out._backward = _backward

        return out

    def __rmul__(self, other):
        return self*other

    def __pow__(self, other):
        assert isinstance(other, (int, float))
        out = Value(self.data ** other, (self, ), f"**{other}")

        def _backward():
            self.grad += other * self.data ** (other - 1) * out.grad

        out._backward = _backward

        return out
    
    def __truediv__(self, other):
        return self * other**-1
    
    def __neg__(self):
        return self*-1
    
    def __sub__(self, other):
        return self + (-other)
    
    def relu(self):
        out = Value(0 if self.data < 0 else self.data, (self, ), 'ReLU')

        def _backward():
            self.grad += (out.data > 0) * out.grad

        out._backward = _backward

        return out

    def tanh(self):
        n = self.data
        t = (math.exp(2 * n) - 1) / (math.exp(2 * n) + 1)
        out = Value(t, (self, ), 'tanh')

        def _backward():
            self.grad += (1 - t**2) * out.grad
        out._backward = _backward
        return out

    def exp(self):
        x = self.data
        out = Value(math.exp(x), (self, ), 'exp')

        def _backward():
            self.grad += out.data * out.grad

        out._backward = _backward
        return out

    def backward(self):
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)  
        build_topo(self)

        self.grad = 1.0

        for node in reversed(topo):
            node._backward()

In [3]:
from graphviz import Digraph

def trace(root):
    nodes, edges = set(), set()

    def build(v):
        if v not in nodes:
            nodes.add(v)
            for child in v._prev:
                edges.add((child, v))
                build(child)

    build(root)
    return nodes, edges


def draw_dot(root, format="svg", rankdir="LR"):
    """
    format: png | svg | ...
    rankdir: TB (top to bottom graph) | LR (left to right)
    """
    assert rankdir in ["LR", "TB"]
    nodes, edges = trace(root)
    dot = Digraph(
        format=format, graph_attr={"rankdir": rankdir}
    )  # , node_attr={'rankdir': 'TB'})

    for n in nodes:
        dot.node(
            name=str(id(n)),
            label="{ %s | data %.4f | grad %.4f}" % (n.label, n.data, n.grad),
            shape="record",
        )
        if n._op:
            dot.node(name=str(id(n)) + n._op, label=n._op)
            dot.edge(str(id(n)) + n._op, str(id(n)))

    for n1, n2 in edges:
        dot.edge(str(id(n1)), str(id(n2)) + n2._op)

    return dot


In [136]:
class Nueron:
    def __init__(self, nin):
        self.w = [Value(random.uniform(-2, 2)) for n in range(nin)]
        self.b = Value(random.uniform(-1, 1)) 

    def __call__(self, x, apply_relu = True):
        act = sum((wi*xi for wi, xi in zip(self.w, x)), self.b)# <w, x> + b
        if apply_relu:
            out = act.relu()
        else:
            out = act
        return out

    def parameters(self):
        return self.w + [self.b]

class Layer:
    def __init__(self, nin, nout):
        self.neurons = [Nueron(nin) for _ in range(nout)]

    def __call__(self, x, apply_relu = True): #x vector of activations for inputting layer to this layer
        outs = [n(x, apply_relu=apply_relu) for n in self.neurons]
        return outs[0] if len(outs) == 1 else outs
    
    def parameters(self):
        return [p for neuron in self.neurons for p in neuron.parameters()]

class MLP:
    def __init__(self, nin, nouts):
        sz = [nin] + nouts
        self.layers = [Layer(sz[i], sz[i + 1]) for i in range(len(nouts))]
        self.hidden_activations = []

    def __call__(self, x):
        activations = []
        for i, layer in enumerate(self.layers):
            if i == len(self.layers) - 1:
                x = layer(x, apply_relu=False)
            else:
                x = layer(x)
                activations.append(x)

        self.hidden_activations.append(activations)
        return x

    def parameters(self):
        return [p for layer in self.layers for p in layer.parameters()]

In [152]:
n = MLP(2, [2, 1])

XOR Training Data

In [139]:
xs = [
    [0.0, 0.0],
    [0.0, 1.0],
    [1.0, 0.0],
    [1.0, 1.0],
]
ys = [0.0, 1.0, 1.0, 0.0]

Training Loop and Graphing Hidden Feature Space

In [ ]:
for k in range(20):
    for p in n.parameters(): #zero_grad
        p.grad = 0.0
    n.hidden_activations = []
    # forward pass
    ypred = [n(x) for x in xs]

    # Extract and pair hidden activations for each input
    hidden_coordinates_per_epoch = []
    labels = []

    for i, activations in enumerate(n.hidden_activations):
        hidden_activations = [b.data for a in activations for b in a]
        hidden_coordinates = [(hidden_activations[i], hidden_activations[i + 1]) for i in range(0, len(hidden_activations), 2)]
        hidden_coordinates_per_epoch.extend(hidden_coordinates)
        labels.append(xs[i])

    if k % 5 == 0:
        plt.figure()
        for coords, label in zip(hidden_coordinates_per_epoch, labels):
            plt.scatter(coords[0], coords[1], label=f"Input: {label}")
            plt.text(coords[0], coords[1], f"{label}", fontsize=9, ha='right')
        
        plt.title(f"Hidden Layer Coordinates at Epoch {k}")
        plt.xlabel('Hidden Unit 1')
        plt.ylabel('Hidden Unit 2')
        plt.xlim(-5, 5)
        plt.ylim(-5, 5)
        plt.legend(loc='upper right')
        plt.grid(True)
        plt.show()

    loss = sum((yout - ygt) ** 2 for ygt, yout in zip(ys, ypred))
    print(f"loss:{loss.data}")
    # backward pass
    loss.backward()
    # weight update
    for p in n.parameters():
        p.data += -0.1 * p.grad